## Dijkstra算法
一个有向图，问：从出发点到各节点的最短距离，返回表。(可以出现距离正无穷)

分析：
- 先准备一张初始表，A—A的距离为0，A—X的距离为Inf
- 选取初始表中最小的那个值(很明显应该选A-A),以这个A为初始点开始找寻到各节点的最短路径(只跳转一次)，如果这个路径的距离加上自己本身的值是小于表中的值，则更新表。得到表二
- 再从表二中选取最小的值(注意不能选取上一步的点)，以此节点开始跳转一次，计算路径的距离并加上自己本身的值，如果这个值小于表中这个节点的值则更新。
- 重复以上步骤直到最后一个节点被使用(usedSet中包含了所有节点)。
- 按步骤运行可以之后知道，其实usedSet储存了已经被遍历过路径的点了，起点startNote到usedSet中的点最短路径已经被updateMap记录了，不可能会再有其他路径能到useSet中的点距离能小于updateMap记录的距离了，因此可以直接跳过对usedSet中点的更新（不跳过也可以）

In [2]:
## 通过字典来得到权有向图
graph = {
    "A":{"B":5, "C":1},              # 字典，和A相连的是B(5)，C(1)
    "B":{"A":5, "C":2, "D":1},       # 字典，和B相连的是A(5)，C(2),D(1)
    "C":{"A":1, "B":2, "D":4, "E":8},# 字典，和C相连的是A(2)，B(2),D(4),E(8)
    "D":{"B":1, "C":4, "E":3, "F":6},# 字典，和D相连的是B(1)，C(4),E(3),F(8)
    "E":{"C":8, "D":3},              # 字典，和E相连的是C(8)，D(3)
    "F":{"D":6}                      # 字典，和F相连的是D(6)
}
graph.get("A",True)

{'B': 5, 'C': 1}

In [4]:

# 将返回当前updateMap中最小的点以及其基准距离
def getMinNote(updateMap, usedSet):
    '''
    usedSet: set()用于记录使用过的点
    updateMap: 到各节点的距离字典
    '''
    minimun = float("inf") 
    minNote = None 
    for i in updateMap :
       if (i not in usedSet) and (updateMap[i] < minimun):
           minNote = i 
           minimun = updateMap[i]
    return minNote, minimun

def Dijkstra(startNote, graph):
    updateMap = {startNote:0}    #放入初始点
    usedSet = set()  #记录使用过的点
    minNote, distance = getMinNote(updateMap, usedSet)

    while minNote != None:      # 当useSet中已经完成了所有节点，则不用再循环了
        edges = graph[minNote]  # 得到的是一个字典，表示minNote能到各个点的距离{X:int...}
        for note in edges:
            if note not in updateMap : # 如果更新字典中不存在这条边
                updateMap[note] = edges[note] + distance
            else :
                # if条件可以删掉
                if note not in usedSet: # startNote到useSet中的点的最短路径已经被updateMap储存了，不可能有更短的路径了，因此可以不用更新了
                    updateMap[note] = min(distance+edges[note], updateMap[note])

        usedSet.add(minNote)
        minNote, distance = getMinNote(updateMap, usedSet)

    return updateMap

graph = {
    "A":{"B":5, "C":1},              # 字典，和A相连的是B(5)，C(1)
    "B":{"A":5, "C":2, "D":1},       # 字典，和B相连的是A(5)，C(2),D(1)
    "C":{"A":1, "B":2, "D":4, "E":8},# 字典，和C相连的是A(2)，B(2),D(4),E(8)
    "D":{"B":1, "C":4, "E":3, "F":6},# 字典，和D相连的是B(1)，C(4),E(3),F(8)
    "E":{"C":8, "D":3},              # 字典，和E相连的是C(8)，D(3)
    "F":{"D":6}                      # 字典，和F相连的是D(6)
}
Dijkstra("A", graph)

{'A': 0, 'B': 3, 'C': 1, 'D': 4, 'E': 7, 'F': 10}

### 方法二
采用小根堆来存储记录，这样就不用遍历整个需要更新的字典来找到最小的距离了。

小根堆的需求：需要自己能update，ignore，主要到这里需要在小根堆的这个类中创造一个heapMap来记录节点的位置，因为这个堆的每个节点会更改，因此每次更改之后要重新排序。

在python中自带的heap可以创建一个元组堆（value,key），也就是把带数字的元组类型推入堆中，并且我们可以通过识别key来更改value的值，并且仍然是堆。实际上就做到了上述的需求。常见一个堆pqueue用来存放没有被完全更新结束的节点，更新结束的意思是当一个节点从pqueue中取出的时候，其实就意味着这个节点已经被更新完毕了。

In [2]:
from heapq import * 

def Dijkstra(startNote, graph):
    updateMap ={startNote:0}    # 放入初始点
    usedSet = set()  # 记录使用过的点
    pqueue = []     # 用来存放还没被完全更新的节点
    heappush(pqueue, (0, startNote)) 

    while pqueue != []:      #当pqueue为空时意味着已经被更新玩完毕了，则不用再循环了
        distance, minNote = heappop(pqueue)
        edges = graph[minNote]
        for note in edges:
            if note not in updateMap : # 如果updateMap中不存在这条边
                updateMap[note] = edges[note] + distance
                heappush(pqueue, (updateMap[note], note))  # 需要把待更新的点压进pqueue
            else :
                if note not in usedSet: # useSet里面的节点不能被更新
                    updateMap[note] = min(distance+edges[note], updateMap[note])
                    heappush(pqueue, (updateMap[note], note))  # 需要把待更新的点压进pqueue

        usedSet.add(minNote)
    return updateMap
    
graph = {
    "A":{"B":5, "C":1},              # 字典，和A相连的是B(5)，C(1)
    "B":{"A":5, "C":2, "D":1},       # 字典，和B相连的是A(5)，C(2),D(1)
    "C":{"A":1, "B":2, "D":4, "E":8},# 字典，和C相连的是A(2)，B(2),D(4),E(8)
    "D":{"B":1, "C":4, "E":3, "F":6},# 字典，和D相连的是B(1)，C(4),E(3),F(8)
    "E":{"C":8, "D":3},              # 字典，和E相连的是C(8)，D(3)
    "F":{"D":6}                      # 字典，和F相连的是D(6)
}

Dijkstra("A", graph)

{'A': 0, 'B': 3, 'C': 1, 'D': 4, 'E': 7, 'F': 10}

# 动态规划
- 暴力递归为什么暴力是因为有着很多的重复运算，我们可以采用缓存的方式来记录这些重复计算的内容，使得下一次遇见相同递归的时候可以直接调用。(记忆化搜索)
-  关键点在于可变参数有多少，我们就创建多大的缓存表，再根据规则可以得到最终结果。




## 问题1
假设有排成一行N个位置，记为1 ~ N，N一定>=2。如果开始时机器人在其中的M位置上(M一定是1 ~ N中的一个)，如果机器人来到1位置那么下一步只能来到2位置；如果机器人来到N位置下一步只能来到N-1位置；如果机器人来到中间位置，那么下一步可以往左一步也可以往右一步，但是规定K步后，最终能来到P位置(也是1~N中的一个)。

给定四个参数N，M，K，P返回有多少种方法数。

分析：
- 在使用递归计算的时候产生重复计算，因此可以用一个二维列表(注意不能用\[\[-1]\*5]*3来生成矩阵，这种方式无法对指定位置更改数据。)来记录下不同的递归返回值，当再次遇见相同的递归返回值时,可以直接调用。(记忆化搜索)
- 可以根据递归的分析过程抽象出动态规划的转移方程。
- 如果是计算分支数量的递推过程中可以重置参数。主要还原现场

In [ ]:
#K表示还有K步，M表示当前位置移动到的位置，P表示目标位置,N表示一共有多少位置,返回种类数
arr = []
all = []    #记录所有方案
dic = dict() #用于缓存不同的M,K取得的种类数
def process(N, M, K, P):

    #剩余步数为0，且当前位置为P，则记录该次结果
    if K == 0 and M == P:
        dic[(M, K)] = 1
        all.append(arr.copy())
        return dic[(M, K)]
    if K == 0 and M != P:
        dic[(M, K)] = 0
        return dic[(M, K)]

    if M == 1:
        arr.append(2)
        dic[(M, K)] = process(N, 2, K-1, P)
        arr.pop()
        return dic[(M, K)]
    if M == N:
        arr.append(N-1)
        dic[(M, K)] = process(N, N-1, K-1, P)
        arr.pop()
        return dic[(M, K)]

    # 如果计算结果已经在缓存里了，则直接调取(这种情况没法记录路径)
    if (M,K) in dic:
        return dic[(M, K)]

    count = 0
    # 计算左移
    arr.append(M-1)
    count = count + process(N, M-1, K-1, P) 
    arr.pop()
    
    # 计算右移
    arr.append(M+1)
    count = count + process(N, M+1, K-1, P)
    arr.pop()
    dic[(M, K)] = count
    return dic[(M, K)]

process(4, 2, 4, 2)
print(all)

## 问题2(背包问题)
有两个长度均为N的数组,每个位置分别代表该件产品weight和value。现在你有一个背包，可容纳重量为bag，请问在不超过容纳重量的前提下，装下货物的最大价值为多少?

改为动态规划:
- 仅根据递归来更改为动态规划，采用二维表的方式来计算 dict(\[(i,\[0]*10) for i in range(4)])形成4行10列的字典(矩阵)
- 先做出能装下所有情况的二维表
- 一般从最后一行开始填满之后再往上层。
- 动态规划的更改其实就是在一般情况中

In [ ]:
import numpy as np
weights = [20, 5, 8, 4, 11]
values = [17, 8, 14, 45, 12]
bag = 40

# rest指剩余的重量
def dpway(weights, values, bag):
    N = len(weights)
    dp = np.zeros((N+1, bag+1)) #生成N+1行bag+1列的矩阵
    # dp =  dict([(i,[0]*(bag+1)) for i in range(N+1)])#生成N+1行bag+1列的矩阵

    for i in range(N-1, -1,-1):
        for rest in range(bag+1):
            if rest >= weights[i]:
                maxValue1 = values[i] + dp[i+1][rest-weights[i]]
            else:
                maxValue1 = 0 #如果当前重量超过了剩余，则不能加入我当前的重量
            maxValue2 = dp[i+1][rest]
            dp[i][rest] = max(maxValue1, maxValue2)
    print(dp)
    return dp[0][bag]

dpway(weights, values, bag)

## 问题3(字符串转化)
规定1与A对应，2与B对应，3与C对应...那么一个数字字符串比如“111”可以转化为“AAA”、“KA”、“AK”。问给定一个数字字符串，有多少种转化方法

改为动态规划：
- 这是一张一维表
- 可以确定的是最后一个N位置是1
- return位置就是设置bp[]的时候


In [ ]:
arr = "9211234126"
def dpway(arr):
    N = len(arr)
    dp = [0]*(N+1)
    dp[N] = 1

    for i in range(N-1, -1, -1):
        if arr[i] == "0":   #如果数字是0也没办法转化
            dp[i] = 0
        elif arr[i] == "1":
            count = dp[i+1]
            if i+2 <= len(arr): # 保证接下来一定是2个数
                count = count + dp[i+2]
            dp[i] = count

        elif arr[i] == "2": 
            count = dp[i+1]
            if i+2 <= len(arr) and int(arr[i:i+2]) <= 26: 
                count = count + dp[i+2]
            dp[i] = count
        else:
            dp[i] = dp[i+1]
    return dp[0]

dpway(arr)

## 问题4 (范围上尝试模型)
给定一个整型数组arr，代表数值不同的纸牌排成一条线，玩家A和玩家B每次拿走一张纸牌，规定玩家A先拿，玩家B后拿，但是每个玩家每次只能拿走最左边或者最右边的纸牌，玩家A和玩家B都绝顶聪明。请返回最后获胜者的分数

更改为动态规划：
- 互相嵌套的动态规划.
- first是正方形表，second也是正方形表
- 范围上的尝试有一半位置是没有的，因为L>R是无效的
- 相互调用，F的值依赖于S的值，S的值依赖于F的值，所以我们需要交替更新。
- 更新对角线采用while循环，每次都行列++，由于每次F和S的更新都是不是依赖于上一对角线所以可以同步进行。放在一个循环里。

In [ ]:
arr = [20, 5, 8, 4, 11]
def dpway(arr):
    N = len(arr)
    F = dict([(i,[0]*(N)) for i in range(N)])
    S = dict([(i,[0]*(N)) for i in range(N)])
    for i in range(N):
        F[i][i] = arr[i]
        S[i][i] = 0
    for R in range(1, N):
        L = 0
        while(R < N):
            F[L][R] = max(arr[L] + S[L+1][R],
                          arr[R] + S[L][R-1])
            S[L][R] = min(F[L+1][R], F[L][R-1])
            L = L + 1
            R = R + 1
    return max(F[0][N-1], S[0][N-1])
dpway(arr)

## 问题5
给定一个数组arr,每个位置表示面值,且无重复面值,每个可以使用无限次.给定一个指定面额，问用给定数组有多少种方法可以得到该面额？

### 方法一(递归方法)
- 每一张都是用无限次(遍历)，直到rest<0
- 遍历完了就是用下一张。

In [ ]:
arr = [3,10,4,19,2,12,5]
rest = 12
all = []    #记录所有面值张数的组合
ans = []    #记录每次的组合
# index:当前在使用那个位置的面值,剩余rest的钱还需要表示，返回方法数
def process(arr, index, rest, ans):
    if index == len(arr):
        if rest == 0:
            all.append(ans.copy())
            return 1
        else:
            return 0

    count = 0   #用于计数
    zhang = 0   #这一面额的钱的使用数量
    while zhang * arr[index] <= rest :
        ans.append(zhang)
        count += process(arr, index+1, rest-zhang * arr[index], ans)
        ans.pop()
        zhang += 1
        
    return count
print(process(arr, 0, rest, ans))
all

### 方法二(记忆搜索的方法)
- 递归过程总子过程出现了多次重复的情况，因此我们可以利用一个字典来记录子过程的返回值(index,rest):value


In [24]:
arr = [3,10,4,19,2,12,5]
rest = 12
def ways(arr, index, rest):
    N = len(arr)
    dp = dict()
    if index == len(arr):
        if rest == 0:
            dp[(index, rest)] = 1
            return dp[(index, rest)]
        else:
            dp[(index, rest)] = 0
            return dp[(index, rest)]

    #如果在记录里则直接返回递归结果即可
    if (index, rest) in dp:
        return dp[(index, rest)]

    count = 0   #用于计数
    zhang = 0   #这一面额的钱的使用数量
    while zhang * arr[index] <= rest :
        count += ways(arr, index+1, rest-zhang * arr[index])
        zhang += 1
    dp[(index, rest)] = count
    return dp[(index, rest)]

ways(arr, 0, rest)

12

### 方法三(动态规划方法)
- 准备一张二维数组，dp\[index]\[rest] = 次数
- 我们可以填出最后一行
- 发现每一行的值都只和下一行有关(某些指定为位置的数值)
- 发现规律:每一个数据其实可以表示为当前前第x个格子的数值和当前格子下一层的数值有关

In [29]:
arr = [3,10,4,19,2,12,5]
rest = 12

def ways(arr, rest):
    N = len(arr)
    dp = dict([(i,[0]*(rest+1)) for i in range(N+1)])  #此处的维度必须得每个位置+1

    dp[N][0] = 1 
    for index in range(N-1, -1,-1):
        for j in range(rest+1):
            if j >= arr[index]:
                dp[index][j] = dp[index+1][j] + dp[index][j-arr[index]]
            else:
                dp[index][j] = dp[index+1][j]
    return dp[0][rest]
ways(arr, rest)

12